# Análise de Dados com Python - Waffle Charts

**Minicurso:** Análise de Dados com Python

**Instrutor:** Humberto da Silva Neto

**Aluno:** 

---

In [0]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches # needed for waffle Charts

mpl.style.use('bmh') # optional: for ggplot-like style

%matplotlib inline

Um gráfico de waffle é uma visualização interessante que normalmente é criada para exibir o progresso em direção às metas. Geralmente, é uma opção eficaz quando você está tentando adicionar recursos de visualização interessantes a um visual composto principalmente por células, como um painel do Excel.

Para prosseguir, iremos importar uma base de dados$^{[1]}$ de imigrações para o canada entre os anos de 1980 a 2013.

---
$^{[1]}$ Correções já aplicadas

## Importando a tabela

In [0]:
df = pd.read_csv('https://raw.githubusercontent.com/hsneto/mc-adp/master/datasets/canada.csv', index_col=0)
df.head()

Vamos rever os dados referentes aos seguintes países: Albania, Brasil e a Islândia.

In [0]:
# vamos criar um novo dataframe para esses três países
df_abi = df.loc[['Albania', 'Brazil', 'Iceland'], :]
df_abi

## Criando um waffle chart do zero

Infelizmente, ao contrário de R, os  waffle charts não são incorporados em nenhuma das bibliotecas de visualização do Python$^{[1]}$. Portanto, aprenderemos como criá-las do zero.

Sendo assim, seguiremos as seguintes etapas:

1.   Determinar a proporção de cada categoria em relação ao total.
2.   Definir o tamanho total do gráfico de waffle.
3.   Determinar o número de peças em função da proporção de cada categoria.
4.   Criar uma matriz que se assemelhe ao gráfico de waffle e preenchê-la.
5.   Mapear a matriz do gráfico de waffle em um gráfico visual.
6.   Criar blocos para melhorar a visualização.
7.   Crie uma legenda.


---
$^{[1]}$ Pesquisando, eu encontrei uma biblioteca chamada PyWaffle no [stackoverflow](https://stackoverflow.com/questions/41400136/how-to-do-waffle-charts-in-python-square-piechart). Para mais, acesse a página do [github](https://github.com/ligyxy/PyWaffle).

### Etapa 1: Determinar a proporção de cada categoria em relação ao total

In [0]:
# calcular a proporção de cada categoria em relação ao total
total_values = sum(df_abi['Total'])

category_proportions = [float(value)/total_values for value in df_abi['Total']]

# imprime proporções
for i, proportion in enumerate(category_proportions):
  print(df_abi.index.values[i] + ':' + str(proportion))

### Etapa 2: Definir o tamanho total do gráfico de waffle

In [0]:
width = 20
height = 5

num_tiles = height * width

print('número total de peças: ' + str(num_tiles))

### Etapa 3: Determinar o número de peças em função da proporção de cada categoria

In [0]:
# calcula o número de peças para cada categoria
tiles_per_category = [round(proportion * num_tiles) for proportion in category_proportions]

# imprime o número de peças por categoria
for i, tiles in enumerate(tiles_per_category):
    print (df_abi.index.values[i] + ': ' + str(tiles))

Isso significa que para cada 100 pessoas que imigraram para o Canadá entre os anos de 1980 a 2013 (considerando somente esses três países), 65 são brasileiros, 34 albanêses e somente 1 é islandes.

### Etapa 4: Criar uma matriz que se assemelhe ao gráfico de waffle e preenchê-la

In [0]:
# inicializa o gráfico de waffle como uma matriz vazia
waffle_chart = np.zeros((height, width))

# define índices para percorrer o gráfico de waffle
category_index = 0
tile_index = 0

# preencher o gráfico de waffle
for col in range(width):
  for row in range(height):
    tile_index += 1

    # se o número de blocos preenchidos para a categoria atual for igual aos blocos correspondentes alocados ...
    if tile_index > sum(tiles_per_category[0:category_index]):
      # ... prossiga para a próxima categoria
      category_index += 1       
            
    # defina o valor da classe para um inteiro, o que aumenta com a classe
    waffle_chart[row, col] = category_index
        
print ('Waffle chart preenchido!')

Vamos dar uma olhada na matriz gerada

In [0]:
waffle_chart

Como esperado, a matriz consiste em três categorias e o número total de instâncias de cada categoria corresponde ao número total de blocos atribuídos a cada categoria.

Como por exemplo. encontramos apenas um **3.** representando o único imigrante islandes.

### Etapa 5: Mapear a matriz do gráfico de waffle em um gráfico visual

In [0]:
plt.matshow(waffle_chart, cmap=plt.cm.gnuplot)
plt.colorbar()

Caso você tenha interesse em controlar a palheta de cores das palavras plotadas, sinta-se livre para alterar o argumento **colormap**. 

Os links abaixo contém exemplos das palhetas de cores disponíveis:

- [link1](https://matplotlib.org/examples/color/colormaps_reference.html)
- [link2](https://matplotlib.org/users/colormaps.html)

### Etapa 6: Criar blocos para melhorar a visualização

In [0]:
# instanciar um novo objeto de figura
fig = plt.figure()

plt.matshow(waffle_chart, cmap=plt.cm.gnuplot)
plt.colorbar()

ax = plt.gca()
ax.set_xticks(np.arange(-.5, (width), 1), minor=True)
ax.set_yticks(np.arange(-.5, (height), 1), minor=True)
    
# adicione linhas de grade
ax.grid(which='minor', color='w', linestyle='-', linewidth=2)

plt.xticks([])
plt.yticks([])

### Etapa 7: Crie uma legenda

In [0]:
# instanciar um novo objeto de figura
fig = plt.figure()

plt.matshow(waffle_chart, cmap=plt.cm.gnuplot)
plt.colorbar()

ax = plt.gca()
ax.set_xticks(np.arange(-.5, (width), 1), minor=True)
ax.set_yticks(np.arange(-.5, (height), 1), minor=True)
    
# adicione linhas de grade
ax.grid(which='minor', color='w', linestyle='-', linewidth=2)

plt.xticks([])
plt.yticks([])

# para calcular a cor do bloco na legenda
total_rows = df_abi['Total'].shape[0] - 1

# criar legenda
legend_handles = []
for i, category in enumerate(df_abi.index.values):
    label_str = category + ' (' + str(df_abi['Total'][i]) + ')'
    color_val = plt.cm.gnuplot(float(i)/total_rows)
    legend_handles.append(mpatches.Patch(color=color_val, label=label_str))

# adicionar legenda no gráfico
plt.legend(handles=legend_handles,
           loc='lower center', 
           ncol=len(df_abi.index.values),
           bbox_to_anchor=(0., -0.2, 0.95, .1)
          )

## Empacotando o código em uma função

In [0]:
def waffle_charts(data, width=20, height=5, cmap=plt.cm.coolwarm, colorbar=True, path=None):
  def category_proportion(data):
    # calcular a proporção de cada categoria em relação ao total
    total_values = sum(data)
    return [float(value)/total_values for value in data]
  
  def tiles_per_category(category_prop, num_tiles):
    # calcula o número de peças para cada categoria
    return [round(proportion * num_tiles) for proportion in category_prop]

  def populate_waffle_chart(tiles_per_cat, width, height):
    # inicializa o gráfico de waffle como uma matriz vazia
    waffle_chart = np.zeros((height, width))

    # define índices para percorrer o gráfico de waffle
    category_index = 0
    tile_index = 0
    
    # preencher o gráfico de waffle
    for col in range(width):
      for row in range(height):
        tile_index += 1

        # se o número de blocos preenchidos para a categoria atual for igual aos blocos correspondentes alocados ...
        if tile_index > sum(tiles_per_cat[0:category_index]):
          # ... prossiga para a próxima categoria
          category_index += 1       

        # defina o valor da classe para um inteiro, o que aumenta com a classe
        waffle_chart[row, col] = category_index
    
    return waffle_chart
  
  
  cp = category_proportion(data)
  num_tiles = height * width
  tiles_cat = tiles_per_category(cp, num_tiles)
  waffle_chart = populate_waffle_chart(tiles_cat, width, height)
   
  # instanciar um novo objeto de figura
  fig = plt.figure()

  plt.matshow(waffle_chart, cmap=cmap)
  if colorbar:
    plt.colorbar()

  ax = plt.gca()
  ax.set_xticks(np.arange(-.5, (width), 1), minor=True)
  ax.set_yticks(np.arange(-.5, (height), 1), minor=True)

  # adicione linhas de grade
  ax.grid(which='minor', color='w', linestyle='-', linewidth=2)

  plt.xticks([])
  plt.yticks([])

  # para calcular a cor do bloco na legenda
  total_rows = data.shape[0] - 1

  # criar legenda
  legend_handles = []
  for i, category in enumerate(data.index.values):
      label_str = category + ' (' + str(data[i]) + ')'
      color_val = cmap(float(i)/total_rows)
      legend_handles.append(mpatches.Patch(color=color_val, label=label_str))

  # adicionar legenda no gráfico
  lgd = plt.legend(handles=legend_handles,
                   loc='lower center', 
                   ncol=len(data.index.values),
                   bbox_to_anchor=(0., -0.2, 0.95, .1)
                  )
  
  if path is not None:
    plt.savefig(path, dpi=400, bbox_inches='tight', bbox_extra_artist=[lgd])

In [0]:
waffle_charts(df[:10]['Total'], cmap=plt.cm.plasma, colorbar=False)

## Utilizando PyWaffle

In [0]:
!pip install pywaffle

In [0]:
from pywaffle import Waffle

data = {'Albania': 34, 'Brazil': 65, 'Iceland': 1}

fig = plt.figure(
    FigureClass=Waffle, 
    rows=5, 
    values=data,
    colors=("#232066", "#983D3D", "#DCB732"),
    legend={'loc': 'upper left', 'bbox_to_anchor': (1, 1)},
    icons='child', icon_size=18, 
    icon_legend=True
)